### GROUP 19 - Sentiment Training

In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()
df = pd.read_csv("reddit_data.csv")
df.head()

Saving reddit_data.csv to reddit_data.csv


,Post
0,Changes to r/RandomThoughts are coming Hey fri...
1,Pretend it's Monday...I didn't lie 😅 We have p...
2,"The word ""dick"" sounds small and the word ""coc..."
3,How many pillows do you have on your bed? Incl...
4,What's the longest time you've gone without sl...


In [ ]:
#create id column for sentiment scoring after
df["id"] = df.index
df.head()

,Post,id
0,Changes to r/RandomThoughts are coming Hey fri...,0
1,Pretend it's Monday...I didn't lie 😅 We have p...,1
2,"The word ""dick"" sounds small and the word ""coc...",2
3,How many pillows do you have on your bed? Incl...,3
4,What's the longest time you've gone without sl...,4


In [ ]:
df.shape

(912, 2)

In [ ]:
#importing RoBERTa model for sentiment training
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

In [ ]:
#Creating function to assign sentiment score for a post
# 0 -> Negative 1-> Neutral 2 -> Positive
def sentiment(example):
  text = tokenizer(example, return_tensors='pt')
  result = model(**text)
  return int(torch.argmax(result.logits))

In [ ]:
#Assigning a sentiment score for each post in the dataframe
from tqdm.notebook import tqdm
res={}
for i, row in tqdm(df.iterrows(), total= len(df)):
  try:
    text = row['Post']
    id = row['id']

    res[id] = sentiment(text)
  except RuntimeError:
    print(f'Error at {i}')


  0%|          | 0/912 [00:00<?, ?it/s]

Error at 274
Error at 373
Error at 374


In [ ]:
#Building the new Dataframe
new_df = pd.DataFrame.from_dict(res, orient='index')
new_df = new_df.reset_index().rename(columns={'index': 'id', 0: 'Sentiment'})
new_df = new_df.merge(df, how='left')
new_df.head(30)

,id,Sentiment,Post
0,0,1,Changes to r/RandomThoughts are coming Hey fri...
1,1,1,Pretend it's Monday...I didn't lie 😅 We have p...
2,2,0,"The word ""dick"" sounds small and the word ""coc..."
3,3,1,How many pillows do you have on your bed? Incl...
4,4,0,What's the longest time you've gone without sl...
5,5,0,"I miss the name ""Dick"". It's a great boys' nam..."
6,6,0,saying “cock” instead of “dick” is so cringe /...
7,7,1,Do you pee in the shower while showering?
8,8,0,"Nobody will love you, unless you are usable."
9,9,0,I have the most sensitive nipples in the world...


In [ ]:
new_df.shape

(909, 3)

In [ ]:
#save dataframe
new_df.to_csv('sentiment_data.csv', index=False)